<a href="https://colab.research.google.com/github/hank199599/deep_learning_keras_log/blob/main/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST 影像訓練卷積神經網路

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

model = models.Sequential()

model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

## 參數的算法
參數 = 過濾器長*寬*輸入通道數*過濾器個數+過濾器個數

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


最後一個Conv2D層輸出的2D張量必須送到接觸的分類器神經網路  
故需要將3D張量展平為1D張量，才能進行傳遞

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [ ]:
from keras import layers
from keras import models


(train_images,train_labels),(test_images,test_labels) = mnist.load_data()

train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_images,train_labels,epochs=5,batch_size=64)

Epoch 1/5
938/938 [==============================] - 44s 46ms/step - loss: 0.4381 - accuracy: 0.8591
Epoch 2/5
938/938 [==============================] - 43s 46ms/step - loss: 0.0572 - accuracy: 0.9824
Epoch 3/5
938/938 [==============================] - 43s 46ms/step - loss: 0.0359 - accuracy: 0.9892
Epoch 4/5
938/938 [==============================] - 43s 45ms/step - loss: 0.0267 - accuracy: 0.9918
Epoch 5/5
938/938 [==============================] - 42s 45ms/step - loss: 0.0214 - accuracy: 0.9935


運用測試資料來驗證準確度

In [ ]:
test_loss,test_acc = model.evaluate(test_images,test_labels)
test_acc

313/313 [==============================] - 3s 8ms/step - loss: 0.0287 - accuracy: 0.9912


0.9911999702453613

# 卷積層

## 比較
* **密集層**：學習特徵空間中全域的Pattern
* **卷積層**：學習局部的pattern

> 特徵映射圖亦被稱作特徵圖：他本身是一個抽象映射動作  
  
![pic5-1](https://github.com/hank199599/deep_learning_keras_log/blob/main/pictures/5-1.png?raw=true)

## 定義
* 從輸入採樣的區塊大小
* 輸出特徵圖的深度  
　　　　　↓ filter數量　　　　 ↓ 以tuple或list傳入
```python
Conv2D(output_depth,(widow_height,window_width))
```  


## 運作原理
1. 在3D輸入特徵圖上滑動的小窗格，萃取窗格上的3D區塊的特徵  
2. 將每個3D區塊轉換成做張量**shape=(output_depth,)**的1D張量
3. 將所有向量依照空間上的位置排列重新組裝成**shape=(height,width,output_depth)**的3D輸出特徵圖  

> 輸出特徵圖中的每個空間位置和輸入特徵圖中的相同位置相對應  

![pic5-2](https://github.com/hank199599/deep_learning_keras_log/blob/main/pictures/5-2.png?raw=true)

## 可能的問題


### 邊界效應 border effec
![pic5-3](https://github.com/hank199599/deep_learning_keras_log/blob/main/pictures/5-3.png?raw=true)  
透過窗格的滑動萃取特徵圖的同時，  
會使輸出的向量維度發生縮減的情形。

**解決方式**：填補法(padding)  
在輸入特徵圖的每一側增加適當數量的列和行，以便讓每個輸入圖塊都可以讓卷積層的移動窗格掃描到。
```
在Conv2D層中，使用padding參數來設定  
* vaild ；不用填補
* same ： 使用填補方式使輸出輸入具有相同的寬度和高度
```


### 步長(stride)
控制窗格移動的間距  
依此方式進行取樣的卷積層被稱作「**步長設定卷積層**(strided convolutions)」
> 即步長大於1的卷積層


# Maxpooling
從輸入特徵圖中做採樣並輸出樣本的最大值。  
他並非用卷積核(convolution kernel)張量積的方式來轉換的局部區塊


In [8]:
from keras import layers
from keras import models
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model_no_max_pool.add(layers.Conv2D(64,(3,3),activation='relu'))
model_no_max_pool.add(layers.Conv2D(64,(3,3),activation='relu'))

In [9]:
model_no_max_pool.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


# 少量資料從頭訓練
易發生 **overfiting**
* 資料擴增法 (data augmentation)
* 預先訓練神經網路的特徵萃取法 (feature extration with a pretrained network)
* 微調預先訓練神經網路法 (fine-tuning a pretrained network)
[這段操作在本機端上執行]